In [ ]:
DATA_NAME = 'agriVision-full' 
TRANSFORM = 'fourier'
CHANNEL = 'blue'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_red_fourier_full_agriVision.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10,11,...,24,25,26,27,28,29,30,31,32,33
2,1688.90,0.11,-0.09,-0.02,0.03,0.01,-0.00,-0.03,0.00,-0.02,...,-0.00,-0.00,-0.01,0.01,0.00,0.00,-0.00,-0.00,-0.00,0.00
3,0.11,2712.86,-0.01,0.00,0.07,0.04,0.01,0.03,0.02,0.02,...,-0.01,-0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00
4,-0.09,-0.01,1697.32,-0.02,-0.02,-0.00,0.04,-0.01,0.01,0.02,...,-0.01,0.00,0.01,-0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00
5,-0.02,0.00,-0.02,923.89,-0.02,0.01,-0.02,-0.02,-0.02,-0.02,...,0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00
6,0.03,0.07,-0.02,-0.02,582.09,-0.00,-0.00,-0.02,-0.01,-0.01,...,-0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00
7,0.01,0.04,-0.00,0.01,-0.00,383.32,-0.00,-0.00,0.02,0.01,...,0.00,0.01,-0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00
8,-0.00,0.01,0.04,-0.02,-0.00,-0.00,286.58,-0.00,0.01,0.01,...,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00
9,-0.03,0.03,-0.01,-0.02,-0.02,-0.00,-0.00,367.71,-0.00,0.01,...,0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00
10,0.00,0.02,0.01,-0.02,-0.01,0.02,0.01,-0.00,241.04,-0.01,...,0.00,0.00,0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00
11,-0.02,0.02,0.02,-0.02,-0.01,0.01,0.01,0.01,-0.01,256.97,...,0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10,11,...,24,25,26,27,28,29,30,31,32,33
2,1.00000,0.00005,-0.00005,-0.00001,0.00004,0.00001,-0.00000,-0.00003,0.00001,-0.00003,...,-0.00001,-0.00001,-0.00008,0.00006,0.00003,0.00002,-0.00000,-0.00007,-0.00004,0.00002
3,0.00005,1.00000,-0.00001,0.00000,0.00006,0.00004,0.00001,0.00003,0.00003,0.00003,...,-0.00004,-0.00001,-0.00003,0.00002,0.00003,0.00000,-0.00001,-0.00000,-0.00000,0.00003
4,-0.00005,-0.00001,1.00000,-0.00002,-0.00002,-0.00000,0.00006,-0.00002,0.00001,0.00004,...,-0.00006,0.00002,0.00006,-0.00002,-0.00000,-0.00006,0.00002,0.00005,0.00003,-0.00001
5,-0.00001,0.00000,-0.00002,1.00000,-0.00003,0.00002,-0.00004,-0.00003,-0.00004,-0.00004,...,0.00004,0.00001,0.00003,0.00000,-0.00000,-0.00003,-0.00001,-0.00001,0.00011,0.00003
6,0.00004,0.00006,-0.00002,-0.00003,1.00000,-0.00001,-0.00001,-0.00003,-0.00004,-0.00003,...,-0.00002,0.00002,0.00005,-0.00003,0.00002,-0.00000,0.00003,-0.00003,-0.00002,-0.00001
7,0.00001,0.00004,-0.00000,0.00002,-0.00001,1.00000,-0.00001,-0.00000,0.00006,0.00002,...,0.00001,0.00006,-0.00001,0.00005,-0.00005,0.00001,0.00001,0.00002,-0.00002,-0.00003
8,-0.00000,0.00001,0.00006,-0.00004,-0.00001,-0.00001,1.00000,-0.00001,0.00002,0.00005,...,-0.00002,-0.00003,-0.00000,0.00000,-0.00001,-0.00002,0.00003,-0.00001,-0.00001,-0.00005
9,-0.00003,0.00003,-0.00002,-0.00003,-0.00003,-0.00000,-0.00001,1.00000,-0.00001,0.00003,...,0.00004,0.00001,-0.00005,0.00000,-0.00001,0.00002,-0.00002,0.00005,-0.00000,-0.00000
10,0.00001,0.00003,0.00001,-0.00004,-0.00004,0.00006,0.00002,-0.00001,1.00000,-0.00004,...,0.00003,0.00000,0.00002,-0.00005,-0.00000,0.00005,0.00004,-0.00003,-0.00005,0.00005
11,-0.00003,0.00003,0.00004,-0.00004,-0.00003,0.00002,0.00005,0.00003,-0.00004,1.00000,...,0.00001,-0.00001,-0.00001,0.00005,-0.00001,0.00004,-0.00002,0.00003,-0.00000,-0.00002


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.30901138938237843

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[2.30341122e+05 9.25148534e+04 8.65182534e+04 2.66050181e+04
 1.05639129e+04 4.63662958e+03 4.17101346e+03 2.56971684e+03
 2.07337338e+03 1.85663828e+03 1.66217308e+03 1.02590207e+03
 5.49667356e+02 3.65819713e+02 1.71631645e+02 1.07105706e+02
 8.42664946e+01 3.66312484e+01 2.07081245e+01 1.75599999e+01
 1.45360149e+01 7.14378649e+00 3.90199627e+00 1.42195495e+00
 6.83778182e-01 4.91711296e-01 5.85600108e-02 1.47509295e-02
 1.33208791e-03 5.10937986e-04 4.67897142e-05 4.54234146e-26]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,-0.031643,0.998848,-0.032137,-0.011845,-0.006866,-0.004402,-0.003270,-0.004217,-0.002735,-0.002919,...,-0.000134,-0.000293,-0.000051,-0.000053,-0.000008,-0.000016,-2.597507e-06,-1.797334e-06,-5.551968e-07,-1.958360e-07
1,-0.652784,0.003637,0.757528,-0.002707,-0.001387,-0.000834,-0.000581,-0.000788,-0.000502,-0.000519,...,-0.000027,-0.000051,-0.000004,-0.000012,-0.000002,-0.000005,-2.199601e-07,4.561285e-07,1.334573e-07,-1.199134e-07
2,0.756112,0.044106,0.651129,-0.038512,-0.018898,-0.011513,-0.008373,-0.011014,-0.006988,-0.007480,...,-0.000336,-0.000735,-0.000128,-0.000132,-0.000021,-0.000042,-6.255183e-06,-4.658681e-06,-1.464945e-06,-6.289747e-07
3,0.025950,0.012922,0.025723,0.997975,-0.037334,-0.017584,-0.012039,-0.016625,-0.009813,-0.010572,...,-0.000436,-0.000976,-0.000166,-0.000177,-0.000029,-0.000057,-8.690336e-06,-6.161804e-06,-1.014831e-06,-7.330081e-07
4,0.012964,0.007463,0.012931,0.034167,0.996773,-0.041263,-0.022699,-0.037233,-0.017425,-0.019134,...,-0.000695,-0.001536,-0.000260,-0.000283,-0.000044,-0.000088,-1.278139e-05,-9.951375e-06,-2.984419e-06,-1.468807e-06
5,0.004872,0.002932,0.004858,0.010165,0.023349,0.939055,-0.036444,-0.337933,-0.021920,-0.025887,...,-0.000636,-0.001402,-0.000242,-0.000251,-0.000043,-0.000080,-1.183319e-05,-8.588705e-06,-2.847067e-06,-1.496599e-06
6,0.009327,0.005615,0.009278,0.019166,0.041771,0.326467,-0.091265,0.934058,-0.050815,-0.060906,...,-0.001347,-0.003012,-0.000524,-0.000545,-0.000089,-0.000170,-2.652163e-05,-1.732822e-05,-5.694947e-06,-2.841638e-06
7,0.005108,0.003121,0.005055,0.010030,0.018830,0.047821,0.975144,0.055551,-0.093511,-0.157564,...,-0.001234,-0.002758,-0.000470,-0.000495,-0.000081,-0.000157,-2.277421e-05,-1.696173e-05,-5.186447e-06,-3.049446e-06
8,0.003907,0.002381,0.003856,0.007542,0.013639,0.029771,0.102671,0.033217,-0.251306,0.937240,...,-0.001168,-0.002621,-0.000448,-0.000462,-0.000077,-0.000145,-2.334137e-05,-1.500800e-05,-4.771574e-06,-2.843002e-06
9,0.001033,0.000625,0.001016,0.001994,0.003541,0.007160,0.019432,0.008044,0.751908,0.052110,...,-0.000340,-0.000782,-0.000129,-0.000144,-0.000025,-0.000040,-6.051353e-06,-5.709740e-06,-2.170005e-06,-6.355745e-07


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[1.0316426463671213,
 0.996363451074178,
 0.3488712954883664,
 0.002025132469069546,
 0.003226890102456803,
 0.06094481831948917,
 1.0912650729785156,
 0.9444491745448623,
 1.251305601407973,
 0.9478895071483466,
 1.1645588257466144,
 1.0925383643027846,
 1.0415808792481052,
 1.0455815811603772,
 1.078926211758452,
 1.0427203633838547,
 1.0425680743505348,
 1.079821716708065,
 1.039226294420114,
 0.9994046041450801,
 0.9755013263133813,
 1.0568382168241903,
 0.031558799041296326,
 0.9714558840029773,
 1.5889797821291145,
 0.43636040665509057,
 1.141928462488047,
 0.8673509254708918,
 0.07352851311686692,
 0.08857997653664973,
 0.12347367366341622,
 0.11722340055870373]